<a href="https://colab.research.google.com/github/jaejunchoe/HAIDS-Lab/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# Local 문맥을 학습하고 단어 간 가까운 상호작용 정보를 추출.
# Local 문맥에서 단어 간 상호작용을 학습하기 위해 설계된 Attention 레이어.
class LocalAttention(nn.Module):
    def __init__(self, input_size, embed_size, win_size, out_channels):
        super(LocalAttention, self).__init__()                                                                     # 'win_size = 5'를 통해 단어 주변의 문맥 정의.

        self.win_size = win_size
        self.attention_layer = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=(win_size, embed_size)),                                                   # CNN 기반으로 local 문맥의 중요도를 변환하는 Attention_layer
            nn.Sigmoid())                                                                                          # 'win_size = 5'를 고려하여 각 단어의 중요도를 0~1로 표현 -> keypoint: win_size = 5

        self.cnn = nn.Sequential(
            nn.Conv2d(1, out_channels, kernel_size=(1, embed_size)),                                               # Local 문맥을 처리하는 CNN Layer
            nn.Tanh(),
            nn.MaxPool2d((input_size, 1)))                                                                         # 문맥 정보를 집약해 최종적으로 input_size인 10,000으로 Feature map 생성


    ## input data를 padding하고 Attention layer 및 CNN에 전달하여 처리된 output 반환.

    def forward(self, x):
        # Padding dynamically to ensure compatibility
        #padding = torch.zeros(x.size(0), (self.win_size - 1) // 2, x.size(-1))
        padding = torch.zeros(x.size(0), (self.win_size - 1) // 2, x.size(-1), device=x.device)                    # Padding 처리.
        x_pad = torch.cat((padding, x, padding), 1).unsqueeze(1)

        scores = self.attention_layer(x_pad).squeeze(1)                                                            # Attention score 계산: 중요도 계산을 통한 단어별 가중치 적용
        out = torch.mul(x, scores).unsqueeze(1)
        out = self.cnn(out)                                                                                        # 최종적으로 Local 문맥 정보 변환
        return out




# Global 문맥을 학습하고 다양한 길이의 단어 간 상호작용 정보를 추출.
# Global 문맥에서 단어 간 상호작용을 학습하기 위한 Attention 레이어.
class GlobalAttention(nn.Module):
    def __init__(self, input_size, embed_size, out_channels):
        super(GlobalAttention, self).__init__()

        self.attention_layer = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=(input_size, embed_size)),                                                  # Global 단위의 중요도를 계산하는 CNN 기반의 Attention Layer
            nn.Sigmoid())                                                                                           # 입력 데이터의 길이(input_size)를 고려하여 각 단어의 중요도를 0~1로 표현 -> keypoint: 입력 데이터의 길이(input_size)


        ## cnn_1,2,3: Global 문맥의 처리를 위한 CNN Layer
        ## 나머지 구성은 동일함.


        self.cnn_1 = nn.Sequential(
            nn.Conv2d(1, out_channels, kernel_size=(2, embed_size)),
            nn.Tanh(),
            nn.MaxPool2d((input_size - 2 + 1, 1)))
        self.cnn_2 = nn.Sequential(
            nn.Conv2d(1, out_channels, kernel_size=(3, embed_size)),
            nn.Tanh(),
            nn.MaxPool2d((input_size - 3 + 1, 1)))
        self.cnn_3 = nn.Sequential(
            nn.Conv2d(1, out_channels, kernel_size=(4, embed_size)),
            nn.Tanh(),
            nn.MaxPool2d((input_size - 4 + 1, 1)))


    # CNN 필터 크기별로 Global 문맥을 처리하는 함수

    def forward(self, x):
        x = x.unsqueeze(1)
        score = self.attention_layer(x)
        out = torch.mul(x, score)
        return self.cnn_1(out), self.cnn_2(out), self.cnn_3(out)



# 사용자와 아이템 리뷰 데이터를 처리하고, 로컬 및 전역 Attention 메커니즘을 적용한 후 예측 값을 생성하는 메인 모델.
# User와 Item review를 기반으로 Local과 Global 문맥을 결합하고 최종 Rating을 예측.

class CNNDLGA(nn.Module):
    def __init__(self, input_size, embed_size=100, win_size=5, channels_local=200, channels_global=100,
                 hidden_size=500, output_size=50):
        super(CNNDLGA, self).__init__()

        self.localAttentionLayer_user = LocalAttention(input_size, embed_size, win_size, channels_local)
        self.globalAttentionLayer_user = GlobalAttention(input_size, embed_size, channels_global)
        self.localAttentionLayer_item = LocalAttention(input_size, embed_size, win_size, channels_local)
        self.globalAttentionLayer_item = GlobalAttention(input_size, embed_size, channels_global)

        # Fully Connected Layer
        # CNN에서 생성된 Feature map을 고차원 벡터로 변환하는 역할.
        self.fcLayer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),  # 입력 차원을 명시하지 않음
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )



    # LocalAttentionLayer_user와 GlobalAttentionLayer_user를 사용해 문맥 정보를 추출.
    def forward(self, x_user, x_item):
        # User side
        local_user = self.localAttentionLayer_user(x_user)
        global1_user, global2_user, global3_user = self.globalAttentionLayer_user(x_user)
        out_user = torch.cat((local_user, global1_user, global2_user, global3_user), 1).view(x_user.size(0), -1)                # CNN 출력을 연결(torch.cat)하여 Fully Conntected Layer로 전달.
        out_user = self.fcLayer(out_user)

        # Item side
        local_item = self.localAttentionLayer_item(x_item)
        global1_item, global2_item, global3_item = self.globalAttentionLayer_item(x_item)
        out_item = torch.cat((local_item, global1_item, global2_item, global3_item), 1).view(x_item.size(0), -1)                # CNN 출력을 연결(torch.cat)하여 Fully Conntected Layer로 전달.
        out_item = self.fcLayer(out_item)

        # Combine user and item representations
        out = torch.sum(torch.mul(out_user, out_item), 1)                                                                       # 두 출력 벡터를 곱(torch.mul)한 후 합산(torch.sum)하여 최종 rating 예측.
        return out
